In [110]:
import pandas as pd
SRC_DIRNAME = "data"

def load_split(filename):
    return pd.read_csv(
        filename,
        delimiter="\t",
        names=['input', 'output', 'category'])

dataset = {}

for splitname in ("RECOGStrain", "RECOGSdev", "RECOGSgen"):
    dataset[splitname] = load_split(f"{SRC_DIRNAME}/{splitname}.tsv")

In [111]:
test_sentence = dataset["RECOGSgen"]["input"].tolist()[1]
print (test_sentence)

gold_Label = dataset["RECOGSgen"]["output"].tolist()[1]
print (gold_Label)

Zoe thought that a hippo cleaned .
Zoe ( 10 ) ; hippo ( 36 ) ; think ( 3 ) AND agent ( 3 , 10 ) AND ccomp ( 3 , 44 ) AND clean ( 44 ) AND agent ( 44 , 36 )


In [112]:
dspy_recogs_train = [
    dspy.Example(
        sentence=row['input'], logical_form=row['output']
    ).with_inputs("sentence")
    for _, row in dataset['RECOGStrain'].iterrows()]

dspy_recogs_dev = [
    dspy.Example(
        sentence=row['input'], logical_form=row['output']
    ).with_inputs("sentence")
    for _, row in dataset['RECOGSdev'].iterrows()]
    

In [113]:
from datasets import load_dataset
import openai
import os
import dspy
from dotenv import load_dotenv
import spacy

root_path = '.'
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')
# keep the API keys in a `.env` file in the local root directory
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')  # use the .env file as it is a good practice to keep keys outside of one's code

In [114]:
lm = dspy.OpenAI(model='gpt-3.5-turbo', api_key=openai_key)
dspy.settings.configure(lm=lm)

In [115]:
class HierarchyParse(dspy.Signature):
    __doc__ = """Parse a sentence into a hierarchy of phrases."""
    sentence = dspy.InputField(desc="a sentence to parse")
    hierarchy = dspy.OutputField(desc="propose a hierarchy of phrases in the sentence; the hierarchy is marked by opening and closing brackets for each chunk and can be nested; the output should be a string")

class ReCOGS(dspy.Signature):
    __doc__ = """Given a sentence hierarchy, convert it to a ReCOGS logical form"""
    hierarchy = dspy.InputField(desc="a hierarchy of phrases in the sentence")
    logical_form = dspy.OutputField(desc="the ReCOGS logical form of the sentence hierarchy. the output should be a string")

class BasicParse(dspy.Module):
    def __init__(self):
        super().__init__()
        self.parse = dspy.Predict(HierarchyParse)
        self.generaterecogs = dspy.Predict(ReCOGS)

    def forward(self, sentence):
        hierarchy = self.parse(sentence=sentence).hierarchy
        return self.generaterecogs(hierarchy=hierarchy).logical_form

In [116]:
def validate_answer(example, pred, trace=None):
    return example.logical_form.lower() == pred.lower()

In [117]:
from dspy.teleprompt import BootstrapFewShot
from dspy.teleprompt import LabeledFewShot
fewshot_optimizer = LabeledFewShot()
compiled = fewshot_optimizer.compile(student = hierarchy_parse, trainset=dspy_recogs_train)

In [118]:
from dspy.evaluate import Evaluate
# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=dspy_recogs_dev, num_threads=1, display_progress=True, display_table=5)
# Launch evaluation.
evaluator(compiled, metric=validate_answer)

Average Metric: 0 / 46  (0.0):   2%|▏         | 46/3000 [00:26<1:17:29,  1.57s/it]

KeyboardInterrupt: 